In [1]:
!pip install pulp

# Vehicle router problem  with Mixed Integer Programming

## Problem

We have $m$ couriers that must distribute $n ≥ m$ items at different customer locations. Each courier $i$ has a maximum load size $l$i. Each item $j$ has a distribution point $j$ and a size $s_j$ (which can represent for instance a weight or a volume). The goal of VCP is to decide for each courier the items to be distributed and plan a tour (i.e. a sequence of location points to visit) to perform the necessary distribution tasks. Each courier tour must start and end at a given origin point o. Moreover, the maximum load $li$ of the courier $i$ should be respected when items are assigned to it. To achieve a fair division among drivers, the objective is to minimize the maximum distance travelled by any courier.

Instance format An instance of VRP is a text file of the form: 

$
m \newline
n\newline
l_1,l_2,...,l_m\newline
s_1,s_2,...,s_n\newline
D_{1,1},D_{1,2},...D_{1,n+1}\newline
D_{2,1},D_{1,2},...D_{1,n+1}\newline
...\newline
D_{n,1},D_{n,2},...D_{n,n+1}\newline
D_{n+1,1},D_{n+1,2},...D_{n+1,n+1}\newline
$

where $m$, $n$, $li$, $s_j$ have the meaning explained above, while $D_i$,$j$ is the distance between distribution point $i$ and distribution point $j$ for $i,j=1,...,n+1$. In particular, distribution point $n + 1$ corresponds to the origin origin point (clearly, $Di,i=0 for i=1,...,n+1)$. For example, the following input file:


3\
7\
15&nbsp;10&nbsp;&nbsp;7\
3&nbsp;&nbsp;2&nbsp;&nbsp;6&nbsp;&nbsp;8&nbsp;&nbsp;5&nbsp;&nbsp;4&nbsp;&nbsp;4\
0&nbsp;&nbsp;3&nbsp;&nbsp;3&nbsp;&nbsp;6&nbsp;&nbsp;5&nbsp;&nbsp;6&nbsp;&nbsp;6&nbsp;&nbsp;2\
3&nbsp;&nbsp;0&nbsp;&nbsp;4&nbsp;&nbsp;3&nbsp;&nbsp;4&nbsp;&nbsp;7&nbsp;&nbsp;7&nbsp;&nbsp;3\
3&nbsp;&nbsp;4&nbsp;&nbsp;0&nbsp;&nbsp;7&nbsp;&nbsp;6&nbsp;&nbsp;3&nbsp;&nbsp;5&nbsp;&nbsp;3\
6&nbsp;&nbsp;3&nbsp;&nbsp;7&nbsp;&nbsp;0&nbsp;&nbsp;3&nbsp;&nbsp;6&nbsp;&nbsp;6&nbsp;&nbsp;4\
5&nbsp;&nbsp;4&nbsp;&nbsp;6&nbsp;&nbsp;3&nbsp;&nbsp;0&nbsp;&nbsp;3&nbsp;&nbsp;3&nbsp;&nbsp;3\
6&nbsp;&nbsp;7&nbsp;&nbsp;3&nbsp;&nbsp;6&nbsp;&nbsp;3&nbsp;&nbsp;0&nbsp;&nbsp;2&nbsp;&nbsp;4\
6&nbsp;&nbsp;7&nbsp;&nbsp;5&nbsp;&nbsp;6&nbsp;&nbsp;3&nbsp;&nbsp;2&nbsp;&nbsp;0&nbsp;&nbsp;4\
2&nbsp;&nbsp;3&nbsp;&nbsp;3&nbsp;&nbsp;4&nbsp;&nbsp;3&nbsp;&nbsp;4&nbsp;&nbsp;4&nbsp;&nbsp;0\

corresponds to a VCP instance with 3 couriers and 7 items where, e.g., $D_{2n+1}=D_{2,8}=3$ is the distance between the distribution point 2 to $o$, while $D_{4,7}=6$ is the distance between the distribution points 4 and 7. Note that in general $D$ is not guaranteed to be symmetrical, i.e., it is possible that $D_{i,j} ̸=D_{j,i}$. 



# Solution

## Decision variables

### $trips$ variable

3d array of val int with domain 0..1 (binary). The 3d array has size $[1..m,1..n+1,1..n+1]$. Used to represent for each courier trip (first dimension $1..n$) the delivered items (2d array with dimension $(n+1)*(n+1)$. 

To summarize: 
- each courier trip is represented by a $(n+1)\times (n+1)$ matrix of boolean values 0..1.
- each courier matrix identifies the item picked for delivery and points to next item with the same logic defined for input matrix D.
- value 0 for all cells in the same column means courier is not delivering that item (item = column index with base 1).
- value 1 in one cells of the column means courier delivers that itme (item = column index  with base 1) and then proceed to pick next item defined in row=column index.  
- as per input matrix D, row=\textit{n+1} and column=\textit{n+1} identify depots (start and end).
- first item picked by courier is the one with row=\textit{n+1} 
- an example matrix for 2 couriers (Red=courier 1, Green=courier 2) delivering 4 items could look like this (3d array $2 * 5 * 5$):

| | | | | | 
|-|-|-|-|-|
|0|0|<span style="color:red">1</span>|0|0| 
|0|0|0|0|0| 
|0|0|0|0|<span style="color:red">1</span>| 
|<span style="color:red">1</span>|0|0|0|0| 
|0|0|0|<span style="color:red">1</span>|0| 

| | | | | | 
|-|-|-|-|-|
|0|0|0|0|0| 
|0|0|0|0|<span style="color:green">1</span>| 
|0|0|0|0|0| 
|0|0|0|0|0| 
|0|<span style="color:green">1</span>|0|0|0|  

Can be read as: 
- courier \#1 starts from depot and picks item 4 (row=5, value 1 in column 4).  
- courier \#1 then picks item 1 (value 1 in row 4, column 1). 
- courier \#1 then picks item 3 (value 1 in row 1 column 3). 
- courier \#1 then goes to depot (value 1 in row 3 column 5). 
- courier \#2 starts from depot and picks item 2 (row=5, value 1 in column 2).  
- courier \#2 then goes to depot (value 1 in row 2 column 5). 
- courier \#1 matrix has column 2 with all zeroes as doesn't deliver that item. 
- courier \#2 matrix has column 1,3 and 4 with all zeroes as doesn't deliver those items. 

### $courier\_items$ variable

1d array to assign every items to each courier (basing on the load capacity). It has range 1..n (items size) and with values 1..m (couriers size). E.g.

| | | | | | |
|-|-|-|-|-|-|
|3|2|2|1|3|1|

Can be read as: 
* item \#1 is assigned to courier \#3
* item \#2 is assigned to courier \#2
* item \#3 is assigned to courier \#2
* item \#4 is assigned to courier \#1 
* and so on. 

## Objective function

The objective function is the function that calculates the distance travelled by each courier. The problem is to minimize the distance travelled by *any* courier. We can then formalise the objective function as:
$$ obj=\sum_{i=1}^{m}\sum_{j=1}^{n+1}\sum_{k=1}^{n+1}D_{jk}*trips_{ijk}$$ 

## Constraints

### All couriers should pick an item
This constraint is enforced by making sure in decision variable $courier_items$ all couriers are at least assigned one item. Formally:

$$\sum_{i=1}^{m}\sum_{j=1}^{n}(courier\_items_j=i) > 0$$

### Each courier shouldn't carry more than their max load
Each courier has a max load defined in the \textit{couriers} variable. We need to make sure the sum of each item assigned to a courier is not exceeding it's max load: 

$$\sum_{i=1}^{m}\sum_{j=1}^{n}(items_j <= couriers_i\;|\;courier_items_j = i)$$

### All items must be delivered
All items must be delivered, which means that aggregated couriers matrices in $trips$ variable must have exactly one value=1 for each column, except for depot column (index $n+1$). The same for each row: all aggregated couriers matrices must have exactly one value=1 in each row, except for depot row (index $n+1$). Formally: 
$$\sum_{j=1}^{n}(\sum_{i=1}^{m}\sum_{k=1}^{n+1}trips_{ikj}=1 \land\sum_{i=1}^{m}\sum_{k=1}^{n+1}trips_{ijk}=1)$$

### Aggregated courier matrix last row and column must sum $m$ 
Each courier will start from depot (row=$n+1$) and deliver to depot (column=$n+1$), so aggregating all couriers matrices the sum of row=$n+1$ must be $m$, same for 
 the sum of column with index $n+1$:

$$\sum_{i=1}^{m}\sum_{j=1}^{n+1}trips_{ij_{n+1}}= m$$
$$\sum_{i=1}^{m}\sum_{j=1}^{n+1}trips_{i_{n+1}j}= m$$


### The diagonal of each courier matrix must be 0} 
Diagonal should be ignored:

$$\sum_{i=1}^{m}\sum_{j=1}^{n+1}trips_{ijj}=0$$ 


### Each courier matrix columns and rows with same index must have same sum
As visible from the matrices example in the $trips$ variable description, each courier matrix must show rows and columns with same index having the same sum (0 or 1):

$$\sum_{i=1}^{m}\sum_{j=1}^{n+1}(\sum_{k=1}^{n+1}trips_{ikj}=\sum_{k=1}^{n+1}trips_{ijk})$$ 

### Each courier matrix row and columns sum must be less or equal to 1
Each row and column of $trips$ courier matrix must have all single rows and columns (including depots) summing less or equal to 1:

$$ \sum_{j=1}^{n+1}(\sum_{i=1}^{m}\sum_{k=1}^{n+1}trips_{ikj}<=1)$$

### Avoid inner loops
Avoid creating loops between items: symmetric cells in matrix $n\times n$ (not depot) must sum $<= 1$: 

$$\sum_{i=1}^{m}\sum_{j=1}^{n}\sum_{k=1}^{n}(trips_{ijk} + trips_{ikj}<=1)$$ 

### Avoid inner loops when courier delivers 1 item
When courier delivers 1 item we have symmetry between depot row and depot column $trips$ variable. When this happens the internal $n\times n$ matrix (depot are excluded) overall sum must be 0:

$$
\sum_{i=1}^{m}\sum_{j=1}^{n}((trips_i_(_n_+_1_)_j + trips_i_j_(_n_+_1_) = 2) + \sum_{k=1}^{n}\sum_{x=1}^{n}trips_i_k_x >= 0) <= 1\;
$$


In [2]:
from pulp import *
import numpy as np
import pulp as pl
import json
import pandas as pd
import numpy as np
import datetime
import time

In [3]:
def get_inst_from_file(inst):
  inst = str(inst).zfill(2)
  inst = f"inst{str(inst)}"
  inst_file_path = f"/kaggle/input/vehicle-router-problem-instances/{inst}.dat"
  instance = pd.read_csv(inst_file_path, delimiter='\t')
  if len(instance) < 5:
      print("Invalid params, expecting 5 parameters (m,n,couriers,items,D)")
      quit()

  m = 0
  n = 0
  couriers = []
  items = []
  D = []

  f =  open(inst_file_path,'r')
  lines = f.readlines()
  f.close()
  line_count = 0
  for line in lines:
      if line_count == 0:
        m = int(line)
      elif line_count == 1:
        n = int(line)
      elif line_count == 2:
        couriers = [int(x) for x in line.split()]
      elif line_count == 3:
        items = [int(x) for x in line.split()]
      elif line_count > 3:
        line = line.replace('\n','')
        D.append( [int(x) for x in line.split()] )
      line_count += 1

  # print('m=',m)
  # print('n=',n)
  # print('couriers=',couriers)
  # print('items=',items)
  # print('D=',D)
  results = {
          "m": m,
          "n": n,
          "couriers": couriers,
          "items": items,
          "D": D,
          }
  return json.dumps(results) 

In [4]:
def run_mip_model(solver_name, inst):
    #print(pl.listSolvers(onlyAvailable=True))
    print(f'loading instance #{inst}...')
    instance = json.loads(get_inst_from_file(inst))

    m = instance['m']
    n = instance['n']
    couriers = instance['couriers']
    items = instance['items']
    D = instance['D']
    prob = LpProblem("Multiple_Courier_Planning", LpMinimize)

    # decision variables
    Trips = LpVariable.dicts('trips',(range(m),range(n+1),range(n+1)), cat='Binary')
    Max = LpVariable('max_distance', lowBound=0, cat='Integer')
    
    # problem
    prob = LpProblem("max_distance", LpMinimize)
    #print(prob)

    # objective function
    prob += Max
    
    #############
    # Constraints
    #############

    # minimize max
    for i in range(m):
        prob += Max >= [Trips[i][j][k] * D[j][k] for j in range(n+1) for k in range(n+1)]

    print('loading constraints...')
    # all couriers must be assigned an item 
    for j in range(n):
        # aggregated columns (except last) sum == 1
        prob += lpSum([Trips[i][k][j] for i in range(m) for k in range(n+1)]) == 1
        # aggregated rows (except last) sum == 1
        prob += lpSum([Trips[i][j][k] for i in range(m) for k in range(n+1)]) == 1
    
    # aggregated last columns must sum m
    prob += lpSum([Trips[i][k][n] for i in range(m) for k in range(n+1)]) == m
    # aggregated last rows must sum m
    prob += lpSum([Trips[i][n][k] for i in range(m) for k in range(n+1)]) == m
    
    # diagonal must be 0
    for i in range(m):
        for j in range(n+1):
            prob += Trips[i][j][j] == 0
    
    for i in range(m):
        for j in range(n+1):
            # all individual courier matrix colummns must sum <= 1
            prob += lpSum([Trips[i][k][j] for k in range(n+1)]) <= 1
            # all individual courier matrix rows must sum <= 1
            prob += lpSum([Trips[i][j][k] for k in range(n+1)]) <= 1
        
    # couriers should be assigned items based on their max load
    for i in range(m):
        prob += lpSum([items[j] * Trips[i][k][j] for j in range(n) for k in range(n+1)]) <= couriers[i]
              
    # # courier trips: each courier matrix column sum must be equivalent to sum of the row with same index (either 0 or 1)
    for i in range(m):
        for j in range(n+1):
                prob += lpSum([Trips[i][k][j] for k in range(n+1)]) == lpSum([Trips[i][j][k] for k in range(n+1)])
    
    # # break subtours unless deliver 1 items only (depot -> item -> depot): n*n matrix sum of symmetric cells must sum < 2
    for i in range(m):
        for j in range(n):
            for k in range(n):
               prob += (Trips[i][j][k] + Trips[i][k][j]) <= 1
    
    # break internal subtours when 1 item delivered: 
    for i in range(m):
        for j in range(n):
            # symmetric depot are equal + sum of all internal cells > 0 == 1
            prob += (((Trips[i][n][j] + Trips[i][j][n]) == 2) + (lpSum([Trips[i][k][x] for k in range(n) for x in range(n)]) >= 1)) <= 1

    #while True:
    print(f'invoking mip solver {solver_name}... ')
    print("\033[32;5mSolving...\033[0m")

    if solver_name == 'cbc':
        solver = pl.getSolver('PULP_CBC_CMD', msg=0)    
    elif solver_name == 'highs':
        solver = pl.getSolver('HiGHS_CMD', msg=0)  
    else:
        raise Exception(f'invalid solver: {solver_name}')
    
    prob.solve(solver)
    #    if LpStatus[prob.status] == "Optimal":
    #        break
       
    #print_sol(Trips,m,n)

    varsdict = {}
    for v in prob.variables():
        varsdict[v.name] = v.varValue
    
    # all couriers distances
    distances = [sum([value(Trips[i][j][k]) * D[j][k] for j in range(n+1) for k in range(n+1)]) for i in range(m) ]

    # max distance consumed
    max_distance = np.max(distances)

    solution_matrix = []
    for i in range(m):
        solution_matrix.append([[int(value(Trips[i][j][k])) for k in range(n+1)] for j in range(n+1)])
    #print(solution_matrix)
    
    np.array([[[value(Trips[i][j][k]) for j in range(n+1)] for k in range(n+1)] for i in range(m)])
    results = {
          "opt": LpStatus[prob.status] == "Optimal", 
          "obj": float(max_distance),
          "trips": translate_solution(solution_matrix,n),
          "solver": solver_name
      }
    
    return results

def translate_solution(solution_matrix,n):
    ret = []
    for item in solution_matrix:
        ret_item = []
        index = list(item[n]).index(1)
        while index != n:
            #print(index)
            ret_item.append(index+1)
            index = list(item[index]).index(1)
        ret.append(ret_item)
    #print(ret)
    return ret

def print_sol(Trips, m, n):
    for i in range(m):
        print('-'*20)
        for j in range(n+1):
            print([int(value(Trips[i][j][k])) for k in range(n+1)])

In [5]:
for i in range(1,5):
    print('-'*10)
    start_time = datetime.datetime.now()
    res = run_mip_model('cbc',i)
    end_time = datetime.datetime.now()
    elapsed_time = end_time - start_time
    
    print(res)
    print(f'Excution time={str(elapsed_time) }')

----------
loading instance #1...
loading constraints...
invoking mip solver cbc... 
Solving...
{'opt': True, 'obj': 14.0, 'trips': [[1, 3, 4], [2, 5, 6]], 'solver': 'cbc'}
Excution time=0:00:00.654563
----------
loading instance #2...
loading constraints...
invoking mip solver cbc... 
Solving...
{'opt': True, 'obj': 226.0, 'trips': [[4], [7], [2], [9, 6, 8], [3, 5], [1]], 'solver': 'cbc'}
Excution time=0:01:25.627193
----------
loading instance #3...
loading constraints...
invoking mip solver cbc... 
Solving...
{'opt': True, 'obj': 12.0, 'trips': [[5, 6, 3], [4, 2], [1, 7]], 'solver': 'cbc'}
Excution time=0:00:00.324319
----------
loading instance #4...
loading constraints...
invoking mip solver cbc... 
Solving...
{'opt': True, 'obj': 220.0, 'trips': [[7], [9], [8], [5], [1], [4], [2, 3], [6, 10]], 'solver': 'cbc'}
Excution time=0:22:19.838183
